Importando Librerías

In [1]:
#!/usr/bin/python
import random
import numpy as np

Tempo: A más actividad más tempo

In [2]:
#Selección Tempo 100bmp -420bpm
def selector_tempo(act):
    act = round(act,3)
    act_max = 0.07
    act_min = 0.006
    _tempo = 100 + (((act - act_min) * (420-100)) / (act_max - act_min))
    return round(_tempo)

Octavas

In [3]:
#Selección de Octavas Para las 2 melodías
def selector_octavas(pal_alegr,pal_trist,e1,e2,pal_total):
    js_max = 0.0375  
    js_min = -0.0009 
    js = (pal_alegr - pal_trist) / pal_total

    _octava  = 4 + round((js - js_min)*(6-4)/(js_max - js_min))

    if e1 == "joy" or e1 == "trust" or e1 == "sadness":
        _octava_m1 =  _octava + 1
    elif e1 == "anger" or e1 == "fear" or e1 == "disgust":
        _octava_m1 =  _octava - 1

    else:
        _octava_m1 = _octava

    if e2 == "joy" or e2 == "trust" or e2 == "sadness":
        _octava_m2 =  _octava + 1
    elif e2 == "anger" or e2 == "fear" or e2 == "disgust":
        _octava_m2 =  _octava - 1

    else:
        _octava_m2 = _octava 

    return _octava,_octava_m1,_octava_m2

Pitch Countour

In [4]:
def pitch_countour(positives,negatives):
    ratio = positives / negatives
    if ratio > 1:
        return 0
    else:
        return 1

Escalas Gregorianas en Modalidad (key) C

In [5]:
#El Modo Jónico (I)
jonico =  ["C","D","E","F","G","A","B"] 
#El Modo Dórico (II)
dorico =  ["C","D","Eb","F","G","A","Bb"] 
#El Modo Frigo (III)
frigio =  ["C","Db","Eb","F","G","Ab","Bb"] 
#El Modo Lidio (IV)
lidio =  ["C","D","E","F#","G","A","B"] 
#El Modo Mixolidio (V)
mixolidio =  ["C","D","E","F","G","A","Bb"] 
#El Modo Eólico (VI)
eolico =  ["C","D","Eb","F","G","Ab","Bb"] 
#El Modo Locrio (VII)
locrio =  ["C","Db","Eb","F","Gb","Ab","Bb"] 

Escalas Mayores

In [6]:
Amaj = ["A","B","C#","D","E","F#","G#"] 
Bmaj = ["B","C#","D#","E","F#","G#","A#"] 
Cmaj = ["C","D","E","F","G","A","B"] 
Dmaj = ["D","E","F#","G","A","B","C#"] 
Emaj = ["E","F#","G#","A","B","C#","D#"] 
Fmaj = ["F","G","A","Bb","C","D","E"] 
Gmaj = ["G","A","B","C","D","E","F#"] 

Escalas Menores

In [7]:
Amin = ["A","B","C","D","E","F","G"] 
Bmin = ["B","C#","D","E","F#","G","A"] 
Cmin = ["C","D","Eb","F","G","Ab","Bb"] 
Dmin = ["D","E","F","G","A","Bb","C"] 
Emin = ["E","F#","G","A","B","C","D"] 
Fmin = ["F","G","Ab","Bb","C","Db","Eb"] 
Gmin = ["G","A","Bb","C","D","Eb","F"] 

Seleccionando Escala 

In [33]:
def selector_escala(e):
    escala = []
    if e == "joy":
        escala = jonico
    elif e == "sadness":
        escala = Cmin
    elif e == "anger":
        escala = frigio
    elif e == "disgust":
        escala = Gmin
    elif e == "fear":
        escala = Dmin 
    elif e == "surprise":
        escala = lidio
    return escala

The Markov matrix in the Harmony Module

In [34]:
Markov_Matrix = [[0,       0.05,   0.07,   0.35,   0.35,   0.08,   0.1],
                 [0.05,    0,      0.05,   0.15,   0.55,   0.2,    0],
                 [0,       0.07,   0,      0.21,   0.07,    0.65,   0],
                 [0.15,    0.15,   0.05,   0,      0.6,    0.05,   0],
                 [0.64,    0.05,   0.05,   0.13,   0,      0.13,   0],
                 [0.06,    0.35,   0.12,   0.12,   0.35,   0,      0],
                 [1,       0,      0,      0,      0,      0,      0]]

Progresión de Acordes: En base a las probabilidades

In [35]:
def progresion(num_bars,e):
    pos = random.randint(0,2)
    grado = Markov_Matrix[pos]
    prog = []
    for i in range (0,num_bars):
        prog.append(pos)
        pos = np.random.choice(np.arange(0, 7), p=grado)
        grado = Markov_Matrix[pos]
    return prog

Construyendo Progresion de Acordes

In [36]:
def acordes(escala,num_bars,e):
    prog = progresion(num_bars,e)
    values = []
    chords = []
    for chord in prog:
        values.append(escala[(chord)% 7])     #Nota base
        values.append(escala[(chord + 2) % 7]) #Su tercera
        values.append(escala[(chord + 4) % 7]) #Su quinta
        chords.append(values)
        values = []
    return chords

Limitando Ritmos

In [64]:
def ritmo_values(e):
    duracion_1 = [1]   
    duracion_2 = [0.5,0.5]  
    duracion_3 = [0.25,0.25,0.25,0.25]
    duracion_4 = [0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125]
    duracion_5 = [0.5,0.5]
    return duracion_1,duracion_2,duracion_3,duracion_4,duracion_5

Módulo del Ritmo

In [51]:
time_signature = 4
def ritmo(densidades,e):
    num_secciones = len(densidades)
    duracion_1,duracion_2,duracion_3,duracion_4,duracion_5 = ritmo_values(e)
    measure = []
    patron_ritmo = []
    for i in range (0,num_secciones):
        if densidades[i] == 1:
            measure = (duracion_1)
        elif densidades[i] == 2:
            measure = (duracion_2)
        elif densidades[i] == 3:
             measure = (duracion_3)
        elif densidades[i] == 4:
             measure = (duracion_4)
        elif densidades[i] == 5:
             measure = (duracion_5)
        patron_ritmo.append(measure)
        assert (np.sum(measure) == 1)
        measure = []
    return patron_ritmo

Módulo Melodía :Ponemos notas a al patrón de ritmo

In [52]:
def melodia(patron_ritmo,acordes,pitch_contour,escala):
    pitch = []
    patron_melodia = []
    it = 0
    r = random.randint(0,2)
    for measure in patron_ritmo:
        pitch.append(acordes[it][0])       
        if len(measure) > 1:
            for nota in range (1,len(measure)):
                if measure[nota] > 0.125: #change
                     pitch.append(acordes[it][r]) #random
                else:
                    if pitch_contour == 0: 
                        pitch.append(escala[(escala.index(pitch[nota-1]) + 1) % 7])
                    else: 
                        pitch.append(escala[(escala.index(pitch[nota-1]) - 1) % 7])
        it += 1
        patron_melodia.append(pitch)
        pitch = []
    return  patron_melodia

Split Densidades

In [53]:
def split_densities(densidad):
    split = []
    desplazamiento = (max(densidad) - min(densidad)) / 5
    for seccion_densidad in densidad: 
        flag = 0
        tmp1 =  min(densidad)
        tmp2 =  min(densidad) + desplazamiento
        if flag != 1 and seccion_densidad >= tmp1 and seccion_densidad < tmp2:
            split.append(1) 
            flag = 1
        else:
            tmp1 = tmp2 + desplazamiento
        if flag != 1 and seccion_densidad >= tmp2 and seccion_densidad < tmp1:
            split.append(2)
            flag = 1
        else:
            tmp2 = tmp1 + desplazamiento
        if flag != 1 and seccion_densidad >= tmp1 and seccion_densidad < tmp2:
            split.append(3)
            flag = 1
        else:
            tmp1 = tmp2 + desplazamiento
        if flag != 1 and seccion_densidad >= tmp2 and seccion_densidad < tmp1:
            split.append(4)
            flag = 1
        else:
            tmp2 = tmp1 + desplazamiento
        if flag != 1 and seccion_densidad >= tmp1 and seccion_densidad <= max(densidad): #No 100 el des encaja
            split.append(5)
            flag = 1
    return split

Unimos Todo: Mano izquierda: Acordes y Mano derecha: Melodía

In [54]:
def modelo(patron_melodia,patron_ritmo,escala,octava,tempo,voz,acordes):
    output = ""
    measure = ""
    bar = ""
    chords = ""
    c = ""
    output += escala + "  "
    output += "V"+str(voz) + " "
    output += "T" + str(tempo) + " "
    
    #Mano izquiera
    for triada in acordes:
        for nota in triada:
            c += nota + str(octava) + "/1" + '+'
        c = c[:-1]
        chords += c + "  "
        c = ""
    #Mano Derecha
    for melodia,ritmo in zip(patron_melodia,patron_ritmo):
        for nota,tiempo in zip(melodia,ritmo):
            measure += nota + str(octava) + "/" + str(tiempo) + "   " 
        bar += str(measure) + str(measure) 
        measure = ""

    right_hand = output + bar
    left_hand = output + chords
    ##print(left_hand) 
    print(right_hand) 
    return right_hand,left_hand

Obtención de emociones del Texto

In [65]:
anger =  1235
fear =  1410
surprise =  1071
sadness =  1255
joy =  1406
disgust =  855


positives =  3310
negatives =  2806
pal_total =  51239

Emociones Predominates en el Texto

In [66]:
e1 = "fear"
e2 = "joy"

Calculando valores adicionales del Libro

In [67]:
activas = (joy/pal_total + anger/pal_total) 
pasivas = sadness/pal_total
act = (activas - pasivas)

Inicialización de Valores

In [68]:
tempo = selector_tempo(act)
o,o1,o2 = selector_octavas(joy,sadness,e1,e2,pal_total)
o2 = o1 + 1
pitch = pitch_countour(positives,negatives)
escala = selector_escala(e1)
chords = acordes(escala,8,e1)
print("Tempo: ",tempo)
print("Octava oe: ",o," Octava e1: ",o1, " Octava e2: ",o2)
print("Escala: ",escala)

Tempo:  205
Octava oe:  4  Octava e1:  3  Octava e2:  4
Escala:  ['D', 'E', 'F', 'G', 'A', 'Bb', 'C']


Evaluando Densidades

In [69]:
de1 =  [0.028, 0.027, 0.03, 0.033, 0.025, 0.034, 0.03, 0.027]
de2 =  [0.024, 0.025, 0.029, 0.026, 0.023, 0.033, 0.03, 0.026]
doe =  [0.425, 0.424, 0.402, 0.419, 0.396, 0.388, 0.386, 0.415]
densidad1 = split_densities(de1)
densidad2 = split_densities(de2)
densidad3 = split_densities(doe)

print(densidad1)
print(densidad2)
print(densidad3)


patron_ritmo = ritmo(densidad1,e1)
patron_melodia = melodia(patron_ritmo,chords,pitch,escala)

patron_ritmo2 = ritmo(densidad2,e1) #e2?
patron_melodia2 = melodia(patron_ritmo2,chords,pitch,escala)

patron_ritmo3 = ritmo(densidad3,e1)
patron_melodia3 = melodia(patron_ritmo3,chords,pitch,escala)

[2, 2, 3, 5, 1, 5, 3, 2]
[1, 2, 3, 2, 1, 5, 4, 2]
[5, 5, 3, 5, 2, 1, 1, 4]


Generación de Melodía (e1) 

In [70]:
right_hand,left_hand = modelo(patron_melodia,patron_ritmo,"",o1,tempo,0,chords)

  V0 T205 F3/0.5   C3/0.5   F3/0.5   C3/0.5   Bb3/0.5   F3/0.5   Bb3/0.5   F3/0.5   G3/0.25   D3/0.25   D3/0.25   D3/0.25   G3/0.25   D3/0.25   D3/0.25   D3/0.25   A3/0.5   E3/0.5   A3/0.5   E3/0.5   D3/1   D3/1   Bb3/0.5   F3/0.5   Bb3/0.5   F3/0.5   A3/0.25   E3/0.25   E3/0.25   E3/0.25   A3/0.25   E3/0.25   E3/0.25   E3/0.25   D3/0.5   A3/0.5   D3/0.5   A3/0.5   


Generación de Melodía (e2) 

In [71]:
right_hand,left_hand = modelo(patron_melodia2,patron_ritmo2,"",o2,tempo,1,chords)

  V1 T205 F4/1   F4/1   Bb4/0.5   F4/0.5   Bb4/0.5   F4/0.5   G4/0.25   D4/0.25   D4/0.25   D4/0.25   G4/0.25   D4/0.25   D4/0.25   D4/0.25   A4/0.5   E4/0.5   A4/0.5   E4/0.5   D4/1   D4/1   Bb4/0.5   F4/0.5   Bb4/0.5   F4/0.5   A4/0.125   Bb4/0.125   C4/0.125   D4/0.125   E4/0.125   F4/0.125   G4/0.125   A4/0.125   A4/0.125   Bb4/0.125   C4/0.125   D4/0.125   E4/0.125   F4/0.125   G4/0.125   A4/0.125   D4/0.5   A4/0.5   D4/0.5   A4/0.5   


Generación de Melodía (oe) 

In [72]:
right_hand,left_hand = modelo(patron_melodia3,patron_ritmo3,"",o,tempo,2,chords)

  V2 T205 F4/0.5   C4/0.5   F4/0.5   C4/0.5   Bb4/0.5   F4/0.5   Bb4/0.5   F4/0.5   G4/0.25   D4/0.25   D4/0.25   D4/0.25   G4/0.25   D4/0.25   D4/0.25   D4/0.25   A4/0.5   E4/0.5   A4/0.5   E4/0.5   D4/0.5   A4/0.5   D4/0.5   A4/0.5   Bb4/1   Bb4/1   A4/1   A4/1   D4/0.125   E4/0.125   F4/0.125   G4/0.125   A4/0.125   Bb4/0.125   C4/0.125   D4/0.125   D4/0.125   E4/0.125   F4/0.125   G4/0.125   A4/0.125   Bb4/0.125   C4/0.125   D4/0.125   


Melodía (2)

In [63]:
print(escala)

['C', 'D', 'Eb', 'F', 'G', 'Ab', 'Bb']


In [ ]:
CAMBIA TUS RITMOS . MUY BIEN PARA :) PERO NO PARA :( LIMITALOSSS
arregla ese tempo !!!!
Arreglar distribucion . Analizar densidades limpiando lematizando todito :)    
Si termina en 8 0 16 . aumentar un measure mas con 1 o un acorde 
Limpia el texto y si densidadades con respecto a num plab o a la cant de emociones OK.
cambiar valores del tempo y octavas por los nuestros :)
Se divide el texto en secciones.
De cada seccion calcular . Cuántas palabras hay con e1 e2
Tendrás un array [20 10 40 50 ... 80]
Si el número es > entonces, más densidad de notas, porque hay más de esa emoción en dicha seccio
ver ideas cel


Coautor?
Aleatorios :(
Validar?
Postcom
Publicar
    andre carlos ponde de leom carballo U sao paulo